In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/SHA256SUMS.txt
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/CHANGELOG.txt
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/LICENSE.txt
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/outputevents.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/procedureevents.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/datetimeevents.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/ingredientevents.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/chartevents.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/inputevents.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/icustays.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/icu/d_items.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/poe.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/d_labitems.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/drgcodes.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/d_icd_diagnoses.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/emar.csv
/kaggle/input/mimic-iv-2-1/mimic-iv-2.1/hosp/omr.csv
/kaggle/in

In [4]:
!pip install -q torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 \
  scikit-learn==1.6.0 pandas numpy tqdm umap-learn --upgrade \
  -f https://download.pytorch.org/whl/torch_stable.html


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 82.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 73.6 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
mkl-umath 0.1.1 requires numpy<1.27.0

In [5]:
import torch
print("✅ CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))


✅ CUDA available: True
GPU: Tesla T4


In [32]:
# ============================================================
# 1) Imports & Config
# ============================================================
import os, math, random, json
import numpy as np, pandas as pd
from tqdm import tqdm
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    normalized_mutual_info_score as NMI, adjusted_rand_score as ARI
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

class CFG:
    SEED = 42
    DATA_PATH = "/kaggle/input/mimic-iv-2-1/mimic-iv-2.1"   # update to your path
    SAVE_DIR = "/kaggle/working"
    MAX_LEN = 512
    BATCH_SIZE = 64
    LR = 2e-4
    EPOCHS = 50
    D_MODEL = 256
    N_LAYERS = 4
    DROPOUT = 0.1
    MASK_PROB = 0.15
    N_CLUSTERS = 8

def set_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(CFG.SEED)


Device: cuda


In [16]:
# ============================================================
# 2) Build preprocessed Orphanet subset directly from raw MIMIC-IV
# ============================================================
import pandas as pd, numpy as np, gc
from collections import Counter

BASE = "/kaggle/input/mimic-iv-2-1/mimic-iv-2.1"
print("📂 Reading MIMIC-IV tables from:", BASE)

# --- Core hospital tables ---
adm = pd.read_csv(f"{BASE}/hosp/admissions.csv",
                  usecols=["subject_id","hadm_id","admittime","dischtime","hospital_expire_flag"],
                  parse_dates=["admittime","dischtime"])
dx = pd.read_csv(f"{BASE}/hosp/diagnoses_icd.csv",
                 usecols=["subject_id","hadm_id","icd_code","icd_version"])
icustays = pd.read_csv(f"{BASE}/icu/icustays.csv",
                       usecols=["subject_id","hadm_id"]).drop_duplicates()

# --- Orphanet codes ---
orph = pd.read_csv("/kaggle/input/orpha-codes/icd10_diseases.csv")
orph["ICDcodes"] = orph["ICDcodes"].astype(str).str.strip().str.upper()
rare_icd = set(orph["ICDcodes"].unique())

# --- Filter diagnoses to Orphanet codes ---
dx["icd_code"] = dx["icd_code"].astype(str).str.strip().str.upper()
rare_dx = dx[dx["icd_code"].isin(rare_icd)]
rare_hadm_ids = rare_dx["hadm_id"].unique()
rare_subjects = rare_dx["subject_id"].unique()
print(f"✅ Orphanet rare-disease subset: {len(rare_subjects)} patients | {len(rare_hadm_ids)} admissions")

# --- Filter admissions to those IDs ---
adm = adm[adm["hadm_id"].isin(rare_hadm_ids)].copy()
adm = adm.dropna(subset=["admittime","dischtime"])
adm["los_days"] = (adm["dischtime"] - adm["admittime"]).dt.total_seconds()/86400.0
adm["los_days"] = adm["los_days"].clip(lower=0.01, upper=60)
adm["los_days_log"] = np.log1p(adm["los_days"])
adm["in_icu"] = adm["hadm_id"].isin(icustays["hadm_id"]).astype(int)
print("Admissions shape:", adm.shape)

# --- Per-patient sequence assembly (simplified) ---
adm = adm.sort_values(["subject_id","admittime"]).reset_index(drop=True)
subjects = adm["subject_id"].unique()
N = len(subjects)

# Use LOS, ICU flag, and temporal order to build simple sequence feature matrix
# For each admission, create [LOS_log, ICU_flag, Expired_flag]
features = ["los_days_log","in_icu","hospital_expire_flag"]
D = len(features)
T_max = adm.groupby("subject_id").size().max()
T_cap = min(60, T_max)   # cap to reasonable sequence length

X = np.zeros((N, T_cap, D), dtype=np.float32)
M = np.zeros((N, T_cap), dtype=np.float32)
Y_ICU = np.zeros(N, dtype=np.float32)
Y_LOS = np.zeros(N, dtype=np.float32)

for i, sid in enumerate(subjects):
    g = adm[adm["subject_id"] == sid].head(T_cap)
    Xi = g[features].to_numpy(dtype=np.float32)
    L = Xi.shape[0]
    X[i, :L, :] = Xi
    M[i, :L] = 1.0
    Y_ICU[i] = g["in_icu"].max()
    Y_LOS[i] = np.log1p(g["los_days"].sum())

print(f"✅ Final tensors built: X {X.shape}, M {M.shape}, ICU mean={Y_ICU.mean():.3f}, LOS mean={Y_LOS.mean():.2f}")
gc.collect()


📂 Reading MIMIC-IV tables from: /kaggle/input/mimic-iv-2-1/mimic-iv-2.1
✅ Orphanet rare-disease subset: 3781 patients | 5923 admissions
Admissions shape: (5923, 8)
✅ Final tensors built: X (3781, 25, 3), M (3781, 25), ICU mean=0.321, LOS mean=1.95


96

In [24]:
# ============================================================
# 5) Fixed PyTorch Dataset for TinyMamba (with x_true return)
# ============================================================
import torch
from torch.utils.data import Dataset, DataLoader

class EHRTinyDataset(Dataset):
    def __init__(self, X, M, mask_prob=0.15):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.M = torch.tensor(M, dtype=torch.float32)
        self.mask_prob = mask_prob

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx].clone()
        mask = self.M[idx].clone()

        # randomly mask some time steps
        mask_rand = (torch.rand_like(mask) < self.mask_prob).float()
        x_masked = x.clone()
        x_masked[mask_rand.bool()] = 0.0  # zero out masked steps

        return x_masked, x, mask  # return both masked & true versions

# Build dataloader
train_ds = EHRTinyDataset(X, M, mask_prob=CFG.MASK_PROB)
train_loader = DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, shuffle=True)

print(f"✅ Dataset ready: {len(train_ds)} patients | batch size {CFG.BATCH_SIZE}")
print("Example batch shapes:")
xb, x_true, m = next(iter(train_loader))
print("x_masked:", xb.shape, "x_true:", x_true.shape, "mask:", m.shape)


✅ Dataset ready: 3781 patients | batch size 64
Example batch shapes:
x_masked: torch.Size([64, 25, 3]) x_true: torch.Size([64, 25, 3]) mask: torch.Size([64, 25])


In [22]:
# ============================================================
# Cell 6) TinyMamba for continuous features (no external deps)
# Encoder: proj (D->d_model) + pos + L TinyMamba blocks -> CLS pool
# Decoder: d_model->D to reconstruct masked steps (MSE loss)
# ============================================================
import math
import torch
import torch.nn as nn

class TinyMamba(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_model)
        self.fc2 = nn.Linear(d_model, d_model)
        self.gate = nn.Sigmoid()
        self.norm = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(0.1)

    def forward(self, x):      # (B, T, d)
        y = self.gate(self.fc1(x)) * self.fc2(x)
        return self.norm(x + self.drop(y))

class EHR_MambaCont(nn.Module):
    def __init__(self, d_in, d_model=256, n_layers=4, max_T=2048):
        super().__init__()
        self.proj = nn.Linear(d_in, d_model)
        self.pos  = nn.Embedding(max_T, d_model)
        self.blocks = nn.ModuleList([TinyMamba(d_model) for _ in range(n_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.dec  = nn.Linear(d_model, d_in)           # decoder for reconstruction

    def forward(self, x, attn_mask=None):
        """
        x: (B, T, D)  continuous features; already zero where masked
        attn_mask: (B, T) 1.0 for observed timesteps, 0.0 for padded (optional)
        """
        B, T, D = x.size()
        pos_ids = torch.arange(T, device=x.device).unsqueeze(0).expand(B, T)
        h = self.proj(x) + self.pos(pos_ids)           # (B,T,d)
        for blk in self.blocks:
            h = blk(h)
        h = self.norm(h)
        # CLS = timestep 0 representation (you already have chronological sorting)
        cls = h[:, 0]                                   # (B, d)
        recon = self.dec(h)                             # (B, T, D)
        return recon, cls


In [33]:
# ============================================================
# Cell 7) Train masked reconstruction on observed timesteps only (fixed)
# ============================================================
import torch
import torch.nn as nn
from tqdm import tqdm

# --- Initialize model ---
d_model = CFG.D_MODEL
n_layers = CFG.N_LAYERS
model = EHR_MambaCont(d_in=X.shape[2], d_model=d_model, n_layers=n_layers, max_T=X.shape[1]).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=CFG.LR, weight_decay=1e-2)
mse = nn.MSELoss(reduction="none")

EPOCHS = CFG.EPOCHS
model.train()

for epoch in range(1, EPOCHS + 1):
    epoch_loss = 0.0
    pbar = tqdm(train_loader, desc=f"[TinyMamba] Epoch {epoch}")
    for xb, x_true, mask in pbar:  # <-- uses tuple output from new dataset
        xb, x_true, mask = xb.to(device), x_true.to(device), mask.to(device)

        opt.zero_grad(set_to_none=True)

        recon, _ = model(xb)  # (B,T,D)

        # compute reconstruction loss only on masked & valid timesteps
        mask_3d = mask.unsqueeze(-1)  # (B,T,1)
        loss_mat = mse(recon, x_true) * mask_3d
        loss = loss_mat.sum() / mask_3d.sum().clamp_min(1.0)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        epoch_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    print(f"✅ Epoch {epoch} | Mean Loss: {epoch_loss / len(train_loader):.4f}")

print("🎯 Training complete.")
torch.cuda.empty_cache()


[TinyMamba] Epoch 1: 100%|██████████| 59/59 [00:00<00:00, 91.85it/s, loss=0.2153]


✅ Epoch 1 | Mean Loss: 0.6683


[TinyMamba] Epoch 2: 100%|██████████| 59/59 [00:00<00:00, 93.84it/s, loss=0.2247]


✅ Epoch 2 | Mean Loss: 0.2628


[TinyMamba] Epoch 3: 100%|██████████| 59/59 [00:00<00:00, 93.92it/s, loss=0.2643]


...

✅ Epoch 48 | Mean Loss: 0.1435


[TinyMamba] Epoch 49: 100%|██████████| 59/59 [00:00<00:00, 96.69it/s, loss=0.1713]


✅ Epoch 49 | Mean Loss: 0.1618


[TinyMamba] Epoch 50: 100%|██████████| 59/59 [00:00<00:00, 97.13it/s, loss=0.1887]

✅ Epoch 50 | Mean Loss: 0.1562
🎯 Training complete.


In [26]:
# ============================================================
# 5) Correct TinyMamba Dataset (final)
# ============================================================
import torch
from torch.utils.data import Dataset, DataLoader

class EHRTinyDataset(Dataset):
    def __init__(self, X, M, mask_prob=0.15):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.M = torch.tensor(M, dtype=torch.float32)
        self.mask_prob = mask_prob

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx].clone()
        mask = self.M[idx].clone()

        # Randomly drop some time steps
        mask_rand = (torch.rand_like(mask) < self.mask_prob).float()
        x_masked = x.clone()
        # expand mask_rand to match feature dimension
        x_masked[mask_rand.bool(), :] = 0.0  

        return x_masked, x, mask  # masked input, true input, mask

# --- dataloader
train_ds = EHRTinyDataset(X, M, mask_prob=CFG.MASK_PROB)
train_loader = DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, shuffle=True, drop_last=True)

print(f"✅ Dataset ready: {len(train_ds)} patients | batch size {CFG.BATCH_SIZE}")
xb, x_true, m = next(iter(train_loader))
print("x_masked:", xb.shape, "| x_true:", x_true.shape, "| mask:", m.shape)


✅ Dataset ready: 3781 patients | batch size 64
x_masked: torch.Size([64, 25, 3]) | x_true: torch.Size([64, 25, 3]) | mask: torch.Size([64, 25])


In [28]:
# --- remove any old version of EHRTinyDataset still in memory ---
del EHRTinyDataset


In [29]:
# ============================================================
# 5) Correct TinyMamba Dataset (final and persistent)
# ============================================================
import torch
from torch.utils.data import Dataset, DataLoader

class EHRTinyDataset(Dataset):
    def __init__(self, X, M, mask_prob=0.15):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.M = torch.tensor(M, dtype=torch.float32)
        self.mask_prob = mask_prob

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx].clone()
        mask = self.M[idx].clone()
        mask_rand = (torch.rand_like(mask) < self.mask_prob).float()

        # expand mask_rand to same shape as x for feature masking
        mask_rand_expanded = mask_rand.unsqueeze(-1).expand_as(x)
        x_masked = x.clone()
        x_masked[mask_rand_expanded.bool()] = 0.0

        return x_masked, x, mask

# --- Build DataLoader ---
train_ds = EHRTinyDataset(X, M, mask_prob=CFG.MASK_PROB)
train_loader = DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, shuffle=True, drop_last=True)

print(f"✅ Dataset ready: {len(train_ds)} patients | batch size {CFG.BATCH_SIZE}")
xb, x_true, m = next(iter(train_loader))
print("x_masked:", xb.shape, "| x_true:", x_true.shape, "| mask:", m.shape)


✅ Dataset ready: 3781 patients | batch size 64
x_masked: torch.Size([64, 25, 3]) | x_true: torch.Size([64, 25, 3]) | mask: torch.Size([64, 25])
